In [6]:
import sys
import os
from dotenv import load_dotenv

load_dotenv()


# 현재 파일의 위치를 기준으로 프로젝트 루트(backend) 경로를 계산하여 추가
root_path = os.path.abspath(os.path.join(os.getcwd(), "..", ".."))
if root_path not in sys.path:
    sys.path.append(root_path)

In [7]:
from datetime import datetime

from fastapi import APIRouter, Depends, HTTPException, status
from sqlalchemy.orm import Session

from app.agents.multi_agent_graph import graph as multi_agent_graph
from app.deps import get_current_user, get_db
from app.models import Conversation, Message, User
from app.schemas import (
    ChatResponse,
    ConversationCreate,
    ConversationOut,
    MessageCreate,
    MessageOut,
)

router = APIRouter(prefix="/conversations", tags=["conversations"])

In [8]:
def _get_conversation(db: Session, user_id: int, conversation_id: int) -> Conversation:

    conversation = (  # -> 채팅방
        db.query(Conversation)
        .filter(Conversation.id == conversation_id, Conversation.user_id == user_id)
        .first()  # -> 대화는 어차피 1개지만 객체로 받기 위해 추가
    )
    if not conversation:
        raise HTTPException(
            status_code=status.HTTP_404_NOT_FOUND, detail="채팅방을 찾을 수 없습니다."
        )
    return conversation

In [9]:
db_gen = get_db()
actual_db_session = next(db_gen)
try:
    # 3. 진짜 세션을 함수에 전달
    conversation = _get_conversation(db=actual_db_session, user_id=1, conversation_id=3)
    all_messages = conversation.messages
finally:
    # 4. 사용 후 닫기
    actual_db_session.close()

In [10]:
all_messages

In [14]:
chat_history = []
        
for msg in all_messages:
        chat_history.append({"role": msg.role, "content": msg.content})

In [15]:
chat_history

[{'role': 'user',
  'content': '5억짜리 집 1채, 10억짜리 집 1채, 20억짜리 집 1채를 가지고 있을 때 세금을 얼마나 내나요?'},
 {'role': 'assistant',
  'content': '요청하신 사례로 먼저 과세표준(주택분)을 정리해 드립니다. (질문에 주신 전제 — 동일 1세대 다주택자, 공정시장가액비율 60% 적용 — 을 사용했습니다.)\n\n1) 공시가격 → 공정시장가액(60% 반영)\n- 공시가격 5억 → 3.0억\n- 공시가격 10억 → 6.0억\n- 공시가격 20억 → 12.0억\n합계(공정시장가액반영) = 21.0억원\n\n2) 과세표준(예시)\n- 예시 A (기본공제 6억원 적용): 과세표준 = 21.0억 − 6.0억 = 15.0억원  \n- 예시 B (기본공제 미적용): 과세표준 = 21.0억원\n\n다음 단계(종합부동산세액 산출)를 위해서는 다음 항목들이 필요합니다.\n- 적용할 종부세율(누진세율 표),\n- 다주택자 가산세율(중과) 여부 및 비율,\n- 지방(교육)세 포함 여부(보통 종부세액의 10%를 지방소득세/교육세로 추가 부과),\n- 적용연도(연도별 세율·공제는 변경됩니다).\n\n원하시는 방식:\n1) 제가 "예시 세율·가산률"을 가정해서 추정 종부세액(국세+지방세 포함)을 계산해 드릴까요? (이 경우 제가 적용할 가정(예: 기본공제 6억 적용/미적용, 중과 가산률 등)을 먼저 제시합니다.)  \n2) 아니면 귀하께서 적용할 세율·가산률·공제액(또는 적용연도)을 알려주시면 그 값으로 계산해 드리겠습니다.\n\n어떤 방법으로 계산해 드릴까요? 원하시면 제가 예시 가정을 제시하여 즉시 추정 금액(숫자)을 계산해 드리겠습니다.'},
 {'role': 'user', 'content': '안녕'},
 {'role': 'assistant', 'content': '안녕하세요! 무엇을 도와드릴까요?'},
 {'role': 'user',
  'content': '5억짜리 집 1채, 10억짜리 집 1채, 

In [ ]:
def get_all_messages(conversation_id: int,
    db: Session = Depends(get_db),
    current_user: User = Depends(get_current_user)):

    conversation = _get_conversation(db, current_user.id, conversation_id)
    
    all_messages = conversation.messages
    return all_messages

In [ ]:
@router.post("/{conversation_id}/messages", response_model=ChatResponse)
def create_message(
    conversation_id: int,
    payload: MessageCreate,
    db: Session = Depends(get_db),
    current_user: User = Depends(get_current_user),
):

    conversation = _get_conversation(db, current_user.id, conversation_id)

    user_message = Message(
        conversation_id=conversation.id, role="user", content=payload.content
    )
    db.add(user_message)

    if not conversation.title or conversation.title == "새 대화":
        conversation.title = payload.content.strip()[:40]

    conversation.updated_at = datetime.utcnow()
    db.flush()

    try:
        all_messages = conversation.messages
        chat_history = []

        for msg in all_messages:
            if msg.id != user_message.id:
                chat_history.append({"role": msg.role, "content": msg.content})

        result = multi_agent_graph.invoke(
            {"query": payload.content, "chat_history": chat_history}
        )
        answer = result.get("answer") or "답변을 생성하지 못했습니다."

    except Exception as exc:
        db.rollback()
        raise HTTPException(
            status_code=status.HTTP_500_INTERNAL_SERVER_ERROR,
            detail=f"Agent error: {exc}",
        ) from exc

    assistant_message = Message(
        conversation_id=conversation.id, role="assistant", content=answer
    )
    db.add(assistant_message)
    conversation.updated_at = datetime.utcnow()

    db.commit()
    db.refresh(conversation)
    db.refresh(user_message)
    db.refresh(assistant_message)

    return ChatResponse(
        conversation=ConversationOut.model_validate(conversation),
        user_message=MessageOut.model_validate(user_message),
        assistant_message=MessageOut.model_validate(assistant_message),
    )

In [ ]:
# Depends(get_db) is only for FastAPI Routes. When testing manually, create the session directly.

db_gen = get_db()
db = next(db_gen)

try:
    # Assuming user_id=1 and conversation_id=3 exist for testing
    test_user_id = 1
    test_convo_id = 3

    conversation = _get_conversation(db=db, user_id=test_user_id, conversation_id=test_convo_id)
    print(f"Conversation found: ID {conversation.id}, Title: {conversation.title}")
except Exception as e:
    print(f"Error: {e}")
finally:
    db.close()